In [ ]:
from PIL import Image

import numpy as np
import asyncio
try:
    import manga_translator
except ImportError:
    '''Ignored'''
from manga_translator.utils.generic import load_image, Quadrilateral
from manga_translator.config import OcrConfig, Ocr, Config
from manga_translator.ocr import dispatch as dispatch_ocr, prepare as prepare_ocr
from manga_translator.detection import dispatch as dispatch_detection, prepare as prepare_detection

In [ ]:
config = Config()
config.translator.target_lang = 'JPN'

verbose = False
device = 'cpu'

await prepare_detection(config.detector.detector)
await prepare_ocr(config.ocr.ocr, device)

In [ ]:
img_rgb, img_alpha = load_image(Image.open('test_ocr_2.png'))

textlines, mask_raw, mask = await dispatch_detection(
    config.detector.detector, img_rgb, config.detector.detection_size, config.detector.text_threshold,
    config.detector.box_threshold,
    config.detector.unclip_ratio, config.detector.det_invert, config.detector.det_gamma_correct, config.detector.det_rotate, config.detector.det_auto_rotate, device, verbose
)

if not textlines:
    raise Exception('No text found!!!!!!!!!!')

textlines = await dispatch_ocr(config.ocr.ocr, img_rgb, textlines, config.ocr, device, verbose)
for textline in textlines:
    print(f'({textline.prob}%) : {textline.text}')

In [ ]:
print(f'pts: {textlines[0].pts}')
print(f'text: {textlines[0].text}')
print(f'prob: {textlines[0].prob}')
print(f'fg_r: {textlines[0].fg_r}')
print(f'fg_g: {textlines[0].fg_g}')
print(f'fg_b: {textlines[0].fg_b}')
print(f'bg_r: {textlines[0].bg_r}')
print(f'bg_g: {textlines[0].bg_g}')
print(f'bg_b: {textlines[0].bg_b}')

h, w, _ = img_rgb.shape
textlines[0].pts = np.array([[0, 0], [w, 0], [w, h], [0, h]], dtype=np.int32)


# np.array([[0, 0], [w, 0], [w, h], [0, h]], dtype=np.int32)

In [ ]:
img_rgb, img_alpha = load_image(Image.open('test_ocr_3.png'))

h, w, _ = img_rgb.shape
textlines = [Quadrilateral(np.array([[0, 0], [w, 0], [w, h], [0, h]], dtype=np.int32), '', 0)]
textlines = await dispatch_ocr(config.ocr.ocr, img_rgb, textlines, config.ocr, device, verbose)

highest_textline = max(textlines, key=lambda x: x.prob)

if highest_textline is not None:
    print(f'({highest_textline.prob}%) : {highest_textline.text}')